In [ ]:
 ## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Using cached swig-4.2.1-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl.metadata (3.6 kB)
Using cached swig-4.2.1-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.9 MB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0
✨🍰✨ Everything looks OK!
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-8ypmw627
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-8ypmw627
  Resolved https://githu

In [ ]:
pip install --upgrade numpy

  Using cached numpy-2.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyportfolioopt 1.5.5 requires numpy<2.0.0,>=1.22.4, but you have numpy 2.1.0 which is incompatible.
scipy 1.12.0 requires numpy<1.29.0,>=1.22.4, but you have numpy 2.1.0 which is incompatible.
stable-baselines3 2.4.0a7 requires numpy<2.0,>=1.20, but you have numpy 2.1.0 which is incompatible.
wrds 3.2.0 requires numpy<1.27,>=1.26, but you have numpy 2.1.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.10/site-packages/pandas_datareader/compat/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  PANDAS_VERSION = LooseVersion(pd.__version__)
/usr/local/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [ ]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
TRAIN_START_DATE = '2019-01-01'
TRAIN_END_DATE = '2022-10-01'
TRADE_START_DATE = '2022-10-01'
TRADE_END_DATE = '2024-03-01'

In [ ]:
my_custom_tickers = ["AAPL", "TSLA", "GOOG", "AMZN", "MSFT"]

In [ ]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list=my_custom_tickers).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (6495, 8)


In [ ]:
df.shape

In [ ]:
df.sort_values(['date','tic'],ignore_index=True).head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,open,high,low,close,volume,tic,day
0,2019-01-02,38.722500,39.712502,38.557499,37.750084,148158800,AAPL,2
1,2019-01-02,73.260002,77.667999,73.046501,76.956497,159662000,AMZN,2
2,2019-01-02,50.828499,52.616001,50.785500,52.233059,30652000,GOOG,2
3,2019-01-02,99.550003,101.750000,98.940002,95.501335,35329300,MSFT,2
4,2019-01-02,20.406668,21.008667,19.920000,20.674667,174879000,TSLA,2


In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (1298, 8)
Successfully added vix


Successfully added turbulence index


In [ ]:
list_ticker = processed["tic"].unique().tolist() #gets the unique ticker symbol from processed
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str)) #   list of dates spanning the entire range in the processed

combination = list(itertools.product(list_date,list_ticker))  #list of all possible combinations of dates and tickers.

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")        #merge new df with processed
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])    #sorts the processed_full  by the "date" and "tic" columns

processed_full = processed_full.fillna(0)    #fills missing places with 0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)  #sort by date

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2019-01-02,AAPL,38.722500,39.712502,38.557499,37.750084,148158800.0,2.0,0.000000,41.187707,30.552264,0.0,-66.666667,100.0,37.750084,37.750084,23.219999,0.0
1,2019-01-02,AMZN,73.260002,77.667999,73.046501,76.956497,159662000.0,2.0,0.000000,41.187707,30.552264,0.0,-66.666667,100.0,76.956497,76.956497,23.219999,0.0
2,2019-01-02,GOOG,50.828499,52.616001,50.785500,52.233059,30652000.0,2.0,0.000000,41.187707,30.552264,0.0,-66.666667,100.0,52.233059,52.233059,23.219999,0.0
3,2019-01-02,MSFT,99.550003,101.750000,98.940002,95.501335,35329300.0,2.0,0.000000,41.187707,30.552264,0.0,-66.666667,100.0,95.501335,95.501335,23.219999,0.0
4,2019-01-02,TSLA,20.406668,21.008667,19.920000,20.674667,174879000.0,2.0,0.000000,41.187707,30.552264,0.0,-66.666667,100.0,20.674667,20.674667,23.219999,0.0
5,2019-01-03,AAPL,35.994999,36.430000,35.500000,33.989887,365248800.0,3.0,-0.084363,41.187707,30.552264,0.0,-66.666667,100.0,35.869986,35.869986,25.450001,0.0
6,2019-01-03,AMZN,76.000504,76.900002,74.855499,75.014000,139512000.0,3.0,-0.043582,78.732355,73.238143,0.0,-66.666667,100.0,75.985249,75.985249,25.450001,0.0
7,2019-01-03,GOOG,52.049999,52.848999,50.703499,50.745255,36822000.0,3.0,-0.033380,53.593230,49.385084,0.0,-66.666667,100.0,51.489157,51.489157,25.450001,0.0
8,2019-01-03,MSFT,100.099998,100.190002,97.199997,91.988014,42579100.0,3.0,-0.078825,98.713261,88.776089,0.0,-66.666667,100.0,93.744675,93.744675,25.450001,0.0
9,2019-01-03,TSLA,20.466667,20.626667,19.825333,20.024000,104478000.0,3.0,-0.014598,21.269516,19.429151,0.0,-66.666667,100.0,20.349334,20.349334,25.450001,0.0


In [ ]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

4725
1765


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
train.tail()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
944,2022-09-30,AAPL,141.279999,143.100006,138.000000,136.685562,124925300.0,4.0,-4.099845,161.531711,140.630389,38.597373,-201.182451,30.213837,154.886223,156.116946,31.620001,3.418651
944,2022-09-30,AMZN,114.080002,116.919998,112.839996,113.000000,59479600.0,4.0,-4.493868,136.972652,108.994350,41.903205,-130.982051,22.497877,125.989333,127.260500,31.620001,3.418651
944,2022-09-30,GOOG,97.730003,99.494003,96.029999,96.040710,26277800.0,4.0,-4.017447,113.466945,93.870114,37.255390,-122.922286,47.271002,106.789811,111.356023,31.620001,3.418651
944,2022-09-30,MSFT,238.289993,240.539993,232.729996,228.952988,35694800.0,4.0,-8.423246,262.369984,223.406524,37.434525,-107.544134,40.136189,250.830834,259.296719,31.620001,3.418651
944,2022-09-30,TSLA,266.149994,275.570007,262.470001,265.250000,67726600.0,4.0,-3.444014,317.907601,260.719401,45.639132,-135.092216,24.757288,288.861668,282.278723,31.620001,3.418651


In [ ]:
trade.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2022-10-03,AAPL,138.210007,143.070007,137.690002,140.888962,114311700.0,0.0,-4.344751,161.704236,139.136505,41.887038,-179.457951,30.803913,153.927840,156.044646,30.10,21.729545
0,2022-10-03,AMZN,113.580002,116.910004,112.449997,115.879997,50941900.0,0.0,-4.363396,136.564705,108.239297,43.999921,-111.277454,23.553444,125.244333,127.266167,30.10,21.729545
0,2022-10-03,GOOG,97.220001,99.970001,97.019997,99.187134,24840000.0,0.0,-3.829290,112.910007,93.490119,40.823799,-97.274076,43.339986,106.163191,111.008610,30.10,21.729545
0,2022-10-03,MSFT,235.410004,241.610001,234.660004,236.660156,28880400.0,0.0,-7.849927,261.341843,222.928633,41.811588,-83.521321,36.198773,249.342820,258.864946,30.10,21.729545
0,2022-10-03,TSLA,254.500000,255.160004,241.009995,242.399994,98363500.0,0.0,-6.439905,322.506144,253.339858,40.925583,-235.564781,39.172474,287.052779,282.139334,30.10,21.729545
1,2022-10-04,AAPL,145.029999,146.220001,144.259995,144.498978,87830100.0,1.0,-4.199138,161.525056,138.481924,44.527467,-118.009533,19.800855,153.220016,156.068235,29.07,3.582071
1,2022-10-04,AMZN,119.889999,123.000000,119.790001,121.089996,62812600.0,1.0,-3.795836,136.214596,108.087405,47.542839,-40.055154,0.667092,124.840000,127.421834,29.07,3.582071
1,2022-10-04,GOOG,101.040001,102.720001,101.040001,102.293602,22580900.0,1.0,-3.390425,112.462452,93.431251,44.072407,-51.307674,23.054859,105.741671,110.773669,29.07,3.582071
1,2022-10-04,MSFT,245.089996,250.360001,244.979996,244.662186,34888400.0,1.0,-6.672950,260.900522,222.940363,45.878339,-28.931055,9.343033,248.396797,258.618040,29.07,3.582071
1,2022-10-04,TSLA,250.520004,257.500000,242.009995,249.440002,109578500.0,1.0,-8.152129,324.920792,248.427209,42.808169,-190.027999,36.132522,285.703668,282.390945,29.07,3.582071


In [ ]:
INDICATORS

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension    #1+2*5 +8*5=51
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 5, State Space: 51


In [ ]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [ ]:
agent = DRLAgent(env = env_train)

if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

In [ ]:
trained_sac = agent.train_model(model=model_sac,
                             tb_log_name='sac',
                             total_timesteps=50000) if if_using_sac else None

In [ ]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

In [ ]:
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=10000) if if_using_a2c else None

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=10000) if if_using_ddpg else None

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=10000) if if_using_ppo else None

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100,
              "buffer_size": 1000000,
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

In [ ]:
trained_td3 = agent.train_model(model=model_td3,
                             tb_log_name='td3',
                             total_timesteps=10000) if if_using_td3 else None


In [ ]:
import os

folder_path = '/content/trained_models'

contents = os.listdir(folder_path)

# Print the contents
for item in contents:
    print(item)

In [ ]:
import os
from stable_baselines3 import TD3, PPO, A2C, DDPG


# Step 2: Save the trained models
save_dir = '/content/trained_models'
os.makedirs(save_dir, exist_ok=True)

if if_using_td3:
    model_td3.save(os.path.join(save_dir, 'td3_model'))
if if_using_ppo:
    model_ppo.save(os.path.join(save_dir, 'ppo_model'))
if if_using_a2c:
    model_a2c.save(os.path.join(save_dir, 'a2c_model'))
if if_using_ddpg:
    model_ddpg.save(os.path.join(save_dir, 'ddpg_model'))
if if_using_sac:
    model_ddpg.save(os.path.join(save_dir, 'sac_model'))

models = {}
if if_using_td3:
    models['td3'] = TD3.load(os.path.join(save_dir, 'td3_model'), env=env_train)
if if_using_ppo:
    models['ppo'] = PPO.load(os.path.join(save_dir, 'ppo_model'), env=env_train)
if if_using_a2c:
    models['a2c'] = A2C.load(os.path.join(save_dir, 'a2c_model'), env=env_train)
if if_using_ddpg:
    models['ddpg'] = DDPG.load(os.path.join(save_dir, 'ddpg_model'), env=env_train)
if if_using_sac:
    models['sac'] = DDPG.load(os.path.join(save_dir, 'sac_model'), env=env_train)

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv

In [ ]:
# Define function to generate predictions from each model
def generate_model_predictions(models, obs):
    model_predictions = {}
    for name, model in models.items():
        action, _ = model.predict(obs)
        model_predictions[name] = action
    return model_predictions

# Define function to aggregate predictions using majority voting
def aggregate_predictions_vote(predictions):
    predictions = predictions.astype(int)
    predictions = np.maximum(predictions, 0)  # Ensure no negative values
    vote_counts = np.apply_along_axis(lambda x: np.bincount(x, minlength=np.max(predictions) + 1), axis=0, arr=predictions)
    return np.argmax(vote_counts, axis=0)

# Define function to evaluate performance of the models using cumulative returns
def evaluate_models(models, test_data, env_kwargs):
    # Initialize the environment with test data
    env_trade = StockTradingEnv(df=test_data, **env_kwargs)

    # Dictionary to store cumulative returns of each model
    cumulative_returns = {}

    # Evaluate each model individually
    for name, model in models.items():
        obs = env_trade.reset()
        total_reward = 0
        done = False
        while not done:
            # Extract the observation from the tuple (assuming it's the first element)
            action, _ = model.predict(obs[0])
            obs, reward, done, _ = env_trade.step([action])  # Wrap action in a list
            total_reward += reward
        cumulative_returns[name] = float(total_reward[0])
        print(f"Cumulative return of {name}: {total_reward[0]}")

    return cumulative_returns

# Example usage (assuming models is a dictionary with your models)
cumulative_returns = evaluate_models(models, trade, env_kwargs)
print(cumulative_returns)


In [ ]:
# Define function to generate predictions from each model
def generate_model_predictions(models, obs):
    model_predictions = {}
    for name, model in models.items():
        action, _ = model.predict(obs)
        model_predictions[name] = action
    return model_predictions

# Define function to aggregate predictions using majority voting
def aggregate_predictions_vote(predictions):
    predictions = predictions.astype(int)
    predictions = np.maximum(predictions, 0)  # Ensure no negative values
    vote_counts = np.apply_along_axis(lambda x: np.bincount(x, minlength=np.max(predictions) + 1), axis=0, arr=predictions)
    return np.argmax(vote_counts, axis=0)

# Define function to evaluate performance of the models using cumulative returns
def evaluate_models(models, test_data, env_kwargs):
    # Initialize the environment with test data
    env_trade = StockTradingEnv(df=test_data, **env_kwargs)

    # Wrap the environment if needed
    if not isinstance(env_trade, DummyVecEnv):
        env_trade = DummyVecEnv([lambda: env_trade])

    # Dictionary to store cumulative returns of each model
    cumulative_returns = {}

    # Evaluate each model individually
    for name, model in models.items():
        obs = env_trade.reset()
        total_reward = 0
        done = False
        while not done:
            action, _ = model.predict(obs)
            obs, reward, done, _ = env_trade.step([action])  # Wrap action in a list for step method
            total_reward += reward
        cumulative_returns[name] = float(total_reward[0])  # Extract scalar value from array
        print(f"Cumulative return of {name}: {total_reward[0]}")

    # Evaluate the bagged model using majority voting for decision making
    obs = env_trade.reset()
    total_reward = 0
    done = False
    while not done:
        predictions = generate_model_predictions(models, obs)
        aggregated_action = aggregate_predictions_vote(np.array(list(predictions.values())))
        obs, reward, done, _ = env_trade.step([aggregated_action])  # Wrap action in a list for step method
        total_reward += reward

    # Print the final decision based on majority voting
    if aggregated_action == 0:
        print("Majority vote: Hold (No action)")
    elif aggregated_action == 1:
        print("Majority vote: Buy")
    elif aggregated_action == 2:
        print("Majority vote: Sell")

    cumulative_returns['bagged'] = float(total_reward[0])  # Extract scalar value from array
    print(f"Cumulative return of bagged model: {total_reward[0]}")

    return cumulative_returns

In [ ]:
cumulative_returns = evaluate_models(models, trade, env_kwargs)

In [ ]:
pip install stable-baselines3 torch


  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0


In [ ]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import A2C  # A3C-like behavior can be achieved using multiple A2C agents asynchronously
from stable_baselines3.common.policies import ActorCriticPolicy
import torch as th
from torch import nn


In [ ]:
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from torch import nn

class CustomGRUPolicy(ActorCriticPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomGRUPolicy, self).__init__(
            *args,
            **kwargs,
            net_arch=[dict(pi=[64, 64], vf=[64, 64])],
            features_extractor_class=GRUFeatureExtractor,
            features_extractor_kwargs=dict(features_dim=128)
        )

class GRUFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim: int = 128):
        super(GRUFeatureExtractor, self).__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[0]
        self.gru = nn.GRU(input_size=n_input_channels, hidden_size=64, num_layers=2, batch_first=True)
        self.fc = nn.Linear(64, features_dim)

    def forward(self, observations):
        x, _ = self.gru(observations.unsqueeze(0))  # Assuming observations are in [batch_size, n_input_channels]
        x = th.relu(x[:, -1, :])  # Use the output of the last time step
        return self.fc(x)


In [ ]:
!pip install stable-baselines3[extra]

import gym # import gym
from stable_baselines3.common.env_util import make_vec_env # import from env_util instead of vec_env

In [ ]:
!pip install gym==0.21.0 # install gym

import gym # import gym

  Using cached gym-0.21.0.tar.gz (1.5 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
# Assuming this is the structure of your StockTradingEnv class
class StockTradingEnv:
    def __init__(self, stock_dim, hmax, initial_amount, num_stock_shares,
                 buy_cost_pct, sell_cost_pct, reward_scaling, state_space,
                 action_space, tech_indicator_list):
        # ... your code ...
        pass

config = {
    'stock_dim': stock_dimension,
    'hmax': 100,
    'initial_amount': 1000000,
    'num_stock_shares':num_stock_shares,
    'buy_cost_pct': buy_cost_list,
    'sell_cost_pct': sell_cost_list,
    'reward_scaling': 1e-4,
    'state_space': state_space,
    'action_space': stock_dimension,
    'tech_indicator_list': INDICATORS
}

env = make_vec_env(lambda: StockTradingEnv(**config), n_envs=4)

ValueError: The environment is of type <class '__main__.StockTradingEnv'>, not a Gymnasium environment. In this case, we expect OpenAI Gym to be installed and the environment to be an OpenAI Gym environment.